In [1]:
# import all necessary libraries
import numpy as np
import pandas as pd
import importlib
import random
from tqdm import tqdm
import datetime
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
from pykrige.ok import OrdinaryKriging

In [3]:
import st_toolkit as geohl
importlib.reload(geohl)

<module 'st_toolkit' from 'C:\\Users\\kka\\Documents\\intern_run\\st_toolkit.py'>

In [4]:
# Set random seed
random.seed(42)
np.random.seed(42)

In [5]:
processed_data = pd.read_pickle('Final\\processed_data.pickle')

In [6]:
processed_data

,own_Index,own_mmsi,own_geom,own_speed,own_course,target_Index,target_mmsi,target_geom,target_speed,target_course,dcpa,tcpa,hr,rel_movement_angle,dist_euclid,speed_r,cri,timestamp_
datetime,,,,,,,,,,,,,,,,,,
2018-07-12 00:00:41,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,10.453580,-0.223472,58.133423,4.937556,10.655910,9.247937,0.000000,1.531354e+09
2018-07-12 00:00:41,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.108515,-0.005242,-101.762145,6.178911,3.116042,41.291932,0.000000,1.531354e+09
2018-07-12 00:00:41,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,10.453580,-0.223472,-58.133423,1.795963,10.655910,9.247937,0.000000,1.531354e+09
2018-07-12 00:00:41,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.943769,0.247131,-159.895568,0.120345,10.479444,39.287023,0.064588,1.531354e+09
2018-07-12 00:00:41,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,3.108515,-0.005242,101.762145,3.037318,3.116042,41.291932,0.000000,1.531354e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-12 23:57:25,4736527,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.583933333333302 37.95276666666671),1.657704,188.961942,4736526,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-1.683891,5.407627,30.636575,4.709079,6.178706,1.099340,0.192224,1.531440e+09
2018-07-12 23:58:25,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,-5.805277,0.584286,-188.998989,0.625693,6.182760,3.640803,0.023216,1.531440e+09
2018-07-12 23:58:25,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-5.805277,0.584286,188.998989,3.767286,6.182760,3.640803,0.035265,1.531440e+09


In [7]:
# obtain array of lat and lon of all own ship samples from processed_data
import re
# import numpy as np
from concurrent.futures import ThreadPoolExecutor
# from tqdm import tqdm

loc_own = processed_data['own_geom']
loc_tar = processed_data['target_geom']

# Define a function to process a single location
def process_location(temp_loc):
    point_string = str(temp_loc)
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", point_string)
    return float(numbers[0]), float(numbers[1])

lat1 =np.array([])
lon1 =np.array([])

# Define the number of threads (adjust as needed)
num_threads = 16

# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Process each location in parallel
    results = list(tqdm(executor.map(process_location, loc_own), total=len(loc_own), desc='Creating location datasets'))

# Extract lat and lon from results
lon1, lat1 = zip(*results)

# Convert to numpy arrays
lon1 = np.array(lon1)
lat1 = np.array(lat1)

# Print a success message
print("Parallel processing completed.")

Creating location datasets: 100%|██████████████████████████████████████████| 264770/264770 [00:01<00:00, 257854.54it/s]


Parallel processing completed.


In [8]:
# obtain array of lat and lon of all target ship samples from processed_data
lat2 =np.array([])
lon2 =np.array([])

# Define the number of threads (adjust as needed)
num_threads = 16

# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Process each location in parallel
    results = list(tqdm(executor.map(process_location, loc_tar), total=len(loc_tar), desc='Creating location datasets'))

# Extract lat and lon from results
lon2, lat2 = zip(*results)

# Convert to numpy arrays
lon2 = np.array(lon2)
lat2 = np.array(lat2)

# Print a success message
print("Parallel processing completed.")

Creating location datasets: 100%|██████████████████████████████████████████| 264770/264770 [00:01<00:00, 248640.60it/s]


Parallel processing completed.


In [10]:
gdfw2_final

,timestamp_,lon,lat,GUST,VIS,WDIRMET
timestamp,,,,,,
2018-07-12 00:00:00,1.531354e+09,23.0497,37.4979,2.741520,24100.0,304.033541
2018-07-12 00:00:01,1.531354e+09,23.0497,37.4979,2.741520,24100.0,304.033541
2018-07-12 00:00:02,1.531354e+09,23.0497,37.4979,2.741520,24100.0,304.033541
2018-07-12 00:00:03,1.531354e+09,23.0497,37.4979,2.741520,24100.0,304.033541
2018-07-12 00:00:04,1.531354e+09,23.0497,37.4979,2.741520,24100.0,304.033541
...,...,...,...,...,...,...
2018-07-12 23:59:56,1.531440e+09,23.9730,37.9854,3.709840,24100.0,343.465432
2018-07-12 23:59:57,1.531440e+09,23.9730,37.9854,3.709932,24100.0,343.464672
2018-07-12 23:59:58,1.531440e+09,23.9730,37.9854,3.710024,24100.0,343.463911


In [ ]:
from enfactor_calc import get_weather_data # obtain the weather data at the exact location and timestamp of the ship
from enfactor_calc import get_enfactors # calculate the enfactor for the ship

In [13]:
# calculate own_enfactors for all own ships
num_threads = 16

own_enfactor_array = []

# Prepare the input data as lists for parallel processing
lat1_list = lat1.tolist()
lon1_list = lon1.tolist()
timestamp_list = processed_data.timestamp_.tolist()
own_course_list = processed_data.own_course.tolist()


# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Process each location in parallel
    for lat, lon, timestamp, own_course in tqdm(zip(lat1_list, lon1_list, timestamp_list, own_course_list), 
                                                   total=len(processed_data), 
                                                   desc='Calculating enfactors'):
        result = executor.submit(get_enfactors, lat, lon, timestamp, own_course, gdfw2_final, [0.28947368, 0.47368421, 0.23684211])
        own_enfactor_array.append(result.result())

print("Enfactors calculation completed.")

Calculating enfactors: 100%|█████████████████████████████████████████████████| 264770/264770 [1:55:29<00:00, 38.21it/s]

Enfactors calculation completed.


In [14]:
# calculate own_enfactors for all target ships
num_threads = 16

target_enfactor_array = []

# Prepare the input data as lists for parallel processing
lat2_list = lat2.tolist()
lon2_list = lon2.tolist()
timestamp_list = processed_data.timestamp_.tolist()
target_course_list = processed_data.target_course.tolist()


# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Process each location in parallel
    for lat, lon, timestamp, target_course in tqdm(zip(lat2_list, lon2_list, timestamp_list, target_course_list), 
                                                   total=len(processed_data), 
                                                   desc='Calculating enfactors'):
        result = executor.submit(get_enfactors, lat, lon, timestamp, target_course, gdfw2_final, [0.28947368, 0.47368421, 0.23684211])
        target_enfactor_array.append(result.result())

print("Enfactors calculation completed.")

Calculating enfactors: 100%|█████████████████████████████████████████████████| 264770/264770 [1:55:39<00:00, 38.15it/s]

Enfactors calculation completed.


In [15]:
# calculate the resultant enfactor using 75:25 weightage ratio for own ship and target ship
enfactor_array = np.array([])
enfactor_array = 0.75 * np.array(own_enfactor_array) + 0.25 * np.array(target_enfactor_array)

In [16]:
enfactor_df = pd.DataFrame(enfactor_array, columns=["Enfactors"])

# Save to a pickle file
enfactor_df.to_pickle('Final\\enfactor_array.pickle')